In [1]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - Periodically Varying Notebook
# Author: R. W. Pfeifle

# Creation Date: 30 May 2023
# Last Revision: 12 January 2024

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [3]:
# graham2015+2015 here


# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
graham2015 = ((Table.read('Tables/Graham2015/table2.dat', readme = 'Tables/Graham2015/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

graham2015['Name1'] = graham2015['Name']
graham2015['Name2'] = "-99" #graham2015['Name']
graham2015['z1'] = graham2015['z']
graham2015['z2'] = -99
graham2015['z1_type'] = "spec"
graham2015['z2_type'] = '-99'

# Now converting the naming convention to RA and Dec and adding some informative columns
graham2015['RA1'] = graham2015['RAh'].astype(str) + ':' + graham2015['RAm'].astype(str) + ':' + graham2015['RAs'].astype(str)
graham2015['Dec1'] = graham2015['DE-'].astype(str) + graham2015['DEd'].astype(str) + ':' + graham2015['DEm'].astype(str) + ':' + graham2015['DEs'].astype(str)

# Converting the coordinates
coordconvert = SkyCoord(ra = graham2015['RA1'], dec = graham2015['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
graham2015['RA1_deg'] = coordconvert.ra.degree
graham2015['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
graham2015['RA2'] = graham2015['RA1']
graham2015['Dec2'] = graham2015['Dec1']

graham2015['RA2_deg'] = graham2015['RA1_deg']
graham2015['Dec2_deg'] = graham2015['Dec1_deg']

# Adding details about the coordinates
graham2015['Equinox1'] = "J2000"
graham2015['Coordinate_waveband1'] = "Optical"
graham2015['Coordinate_Source1'] = "CRTS"

graham2015['System Type'] = 'Binary AGN Candidate / Binary SMBH Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
graham2015['Brightness1'] = -100
graham2015['Brightness_band1'] = "-100"
graham2015['Brightness_type1'] = "-100"

graham2015['Brightness2'] = -100
graham2015['Brightness_band2'] = "-100"
graham2015['Brightness_type2'] = "-100"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
graham2015['Sep'] = -99 # arcseconds

# We need to come back here and calculate the angular separation based on what Graham+ included as the \
# physical separation (which they only did in certain cases...)

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#graham2015['delta_z'] = graham2015['z']-graham2015['z2']
graham2015['dV'] = -99 #(2.99e+5)*((1+graham2015['z'])**2 - (1+graham2015['z2'])**2)/((1+graham2015['z'])**2+(1+graham2015['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
graham2015['Selection Method'] = "Optical Periodicity" #DPSELs
graham2015['Confirmation Method'] = '-99'
graham2015['Paper(s)'] = "Graham+2015"
graham2015['BibCode(s)'] = "2015MNRAS.453.1562G"
graham2015['DOI(s)'] = "https://doi.org/10.1093/mnras/stv1726"

graham2015['Notes'] = ''

# And dropping any columns that we don't need....
graham2015.drop(labels=['RAh','RAm','RAs','DE-','DEd','DEm','DEs','logMBH','LC','Per','logMBH','tinsp','DtGW'],\
              axis=1, inplace=True)

graham2015

# graham lists separations in pc, sop we'll need to convert back to arcseconds

,Name,z,Vmed,r,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,UM 211,1.998,17.38,0.000,UM 211,-99,1.998,-99,spec,-99,0:12:10.9,-1:22:7.6,3.045417,-1.368778,0:12:10.9,-1:22:7.6,3.045417,-1.368778,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
1,UM 234,0.729,17.82,0.011,UM 234,-99,0.729,-99,spec,-99,0:23:3.2,+1:15:33.9,5.763333,1.259417,0:23:3.2,+1:15:33.9,5.763333,1.259417,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
2,SDSS J014350.13+141453.0,1.438,17.68,0.008,SDSS J014350.13+141453.0,-99,1.438,-99,spec,-99,1:43:50.0,+14:14:54.9,25.958333,14.248583,1:43:50.0,+14:14:54.9,25.958333,14.248583,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
3,PKS 0157+011,1.170,18.04,0.000,PKS 0157+011,-99,1.170,-99,spec,-99,2:0:3.9,+1:25:12.6,30.016250,1.420167,2:0:3.9,+1:25:12.6,30.016250,1.420167,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
4,RX J024252.3-232633,0.680,19.01,0.000,RX J024252.3-232633,-99,0.680,-99,spec,-99,2:42:51.9,-23:26:34.0,40.716250,-23.442778,2:42:51.9,-23:26:34.0,40.716250,-23.442778,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
5,US 3204,0.954,18.09,0.008,US 3204,-99,0.954,-99,spec,-99,2:49:28.9,+1:9:25.0,42.370417,1.156944,2:49:28.9,+1:9:25.0,42.370417,1.156944,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
6,CT 638,0.265,16.68,0.000,CT 638,-99,0.265,-99,spec,-99,3:18:6.5,-34:26:37.4,49.527083,-34.443722,3:18:6.5,-34:26:37.4,49.527083,-34.443722,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
7,RXS J04117+1324,0.277,16.20,0.006,RXS J04117+1324,-99,0.277,-99,spec,-99,4:11:46.9,+13:24:16.5,62.945417,13.404583,4:11:46.9,+13:24:16.5,62.945417,13.404583,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
8,HS 0423+0658,0.170,15.73,0.000,HS 0423+0658,-99,0.170,-99,spec,-99,4:26:30.2,+7:5:30.3,66.625833,7.091750,4:26:30.2,+7:5:30.3,66.625833,7.091750,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,
9,2MASS J04352649-1643460,0.098,16.96,0.004,2MASS J04352649-1643460,-99,0.098,-99,spec,-99,4:35:26.5,-16:43:45.7,68.860417,-16.729361,4:35:26.5,-16:43:45.7,68.860417,-16.729361,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-99,-99,Optical Periodicity,-99,Graham+2015,2015MNRAS.453.1562G,https://doi.org/10.1093/mnras/stv1726,


In [4]:
# Adding in Lu+2016 here:

#**********************
# The link to the full table is broken on the MNRAS website. I haved emailed MNRAS about this months ago \
# but it was never fixed and I still cannot acces their full list. We are skipping Liu+2016 for now.
#

#lu2016 = pd.read_csv('Tables/Lu2016/Lu2016.csv', sep=',')
# 
#for index, row in graham2015.iterrows():
#    if row['Name'] in lu2016['Name']:
#        print('True')
#        graham2015.at[index, 'Paper(s)'] += ' ; Lu+2016'
#        graham2015.at[index, 'BibCode(s)'] += ' ; 2016MNRAS.459L.124L' 
#        graham2015.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnrasl/slw062'

        

In [5]:
# Now adding Charisi+2016 here...

charisi2016 = pd.read_csv('Tables/Charisi2016/Charisi2015_t1.csv', sep=',')
charisi2016t2 = pd.read_csv('Tables/Charisi2016/Charisi2015_t2.csv', sep=',')

charisi2016 = charisi2016.merge(charisi2016t2, on='Name', how='left')

# NOTE: I am including all 50 candidates identified by Charisi, but we should go and add a confidence flag later
# specifying which ones remained significant after the additional iPTF and CRTS data was added to extend the baseline

# NOTE: We need to go back and get the sexagesiaml coorddinates here

# NOTE: Charisi quore separations in Mpc but I think they're supposed to be pc...

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

# Charisi has two typos in their degree coordinates. I have 
charisi2016['Name1'] = charisi2016['Name']
charisi2016['Name2'] = "-99"
charisi2016['z1'] = charisi2016['z']
charisi2016['z2'] = -99
charisi2016['z1_type'] = "spec"
charisi2016['z2_type'] = -99

# Now converting the naming convention to RA and Dec and adding some informative columns

## Converting the coordinates
# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=charisi2016['RA']*u.degree, dec=charisi2016['Dec']*u.degree, frame='icrs')
charisi2016['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
charisi2016['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)

# Convert RA and Dec to the desired format
ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)


charisi2016['RA1_deg'] = charisi2016['RA']# .astype(float)
charisi2016['Dec1_deg'] = charisi2016['Dec']#.astype(float)


# Adding in a second set of coordinates for the 'secondary'
charisi2016['RA2'] = charisi2016['RA1']
charisi2016['Dec2'] = charisi2016['Dec1']

charisi2016['RA2_deg'] = charisi2016['RA1_deg']
charisi2016['Dec2_deg'] = charisi2016['Dec1_deg']

# Adding details about the coordinates
charisi2016['Equinox1'] = "J2000"
charisi2016['Coordinate_waveband'] = "-99"
charisi2016['Coordinate_Source'] = "-99"

charisi2016['System Type'] = 'Binary AGN Candidate / Binary SMBH Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
charisi2016['Brightness1'] = -100
charisi2016['Brightness_band1'] = "-100"
charisi2016['Brightness_type1'] = "-100"

charisi2016['Brightness2'] = -100
charisi2016['Brightness_band2'] = "-100"
charisi2016['Brightness_type2'] = "-100"

charisi2016['Sep'] = charisi2016['Angular']*(1.e-6) # arcseconds

### For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
##charisi2016['delta_z'] = charisi2016['z']-charisi2016['z2']
charisi2016['dV'] = -99 #(2.99e+5)*((1+charisi2016['z'])**2 - (1+charisi2016['z2'])**2)/((1+charisi2016['z'])**2+(1+charisi2016['z2'])**2)
### dV will be zero until we include follow-up observations that show separate redshifts
##
# Adding information about the paper and the selection method
charisi2016['Selection Method'] = "Optical Periodicity" #DPSELs
charisi2016['Confirmation Method'] = '-99'
charisi2016['Paper(s)'] = "Charisi+2016"
charisi2016['BibCode(s)'] = "2016MNRAS.463.2145C"
charisi2016['DOI(s)'] = "https://doi.org/10.1093/mnras/stw1838"

charisi2016['Notes'] = ''

# And dropping any columns that we don't need....
charisi2016.drop(labels=['log(M/M_sun)','log(M/M_sun)_err','X-ray/radiocatalogue','delta_t','tinsp','tinsp.1',\
                        'P-value','Observed','Variability'], axis=1, inplace=True)

charisi2016

# graham lists separations in pc, sop we'll need to convert back to arcseconds



,Name,RA,Dec,z,R-mag,Separation,Angular,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,UM269,10.832244,0.854291,0.308,17.8,NaN,NaN,UM269,-99,0.308,-99,spec,-99,0:43:19.74,0:51:15.45,10.832244,0.854291,0:43:19.74,0:51:15.45,10.832244,0.854291,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,NaN,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
1,SDSS J005158.83-002054.1,12.995131,-0.348382,1.047,19.3,2.89,0.35,SDSS J005158.83-002054.1,-99,1.047,-99,spec,-99,0:51:58.83,-0:20:54.18,12.995131,-0.348382,0:51:58.83,-0:20:54.18,12.995131,-0.348382,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,3.500000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
2,SDSS J005453.30-003258.3,13.722104,-0.549532,0.961,19.8,1.55,0.19,SDSS J005453.30-003258.3,-99,0.961,-99,spec,-99,0:54:53.30,-0:32:58.32,13.722104,-0.549532,0:54:53.30,-0:32:58.32,13.722104,-0.549532,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,1.900000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
3,SDSS J023050.06+005843.1,37.708607,0.978662,1.447,17.9,6.65,0.77,SDSS J023050.06+005843.1,-99,1.447,-99,spec,-99,2:30:50.07,0:58:43.18,37.708607,0.978662,2:30:50.07,0:58:43.18,37.708607,0.978662,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,7.700000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
4,SDSS J024442.77-004223.2,41.178244,-0.706457,0.628,19.7,1.77,0.25,SDSS J024442.77-004223.2,-99,0.628,-99,spec,-99,2:44:42.78,-0:42:23.24,41.178244,-0.706457,2:44:42.78,-0:42:23.24,41.178244,-0.706457,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,2.500000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
5,2QZ J095344.7+010354,148.436691,1.065195,0.994,19.3,1.70,0.22,2QZ J095344.7+010354,-99,0.994,-99,spec,-99,9:53:44.81,1:03:54.70,148.436691,1.065195,9:53:44.81,1:03:54.70,148.436691,1.065195,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,2.200000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
6,SDSS J104648.62+513912.6,161.702601,51.653513,0.587,18.7,4.30,0.63,SDSS J104648.62+513912.6,-99,0.587,-99,spec,-99,10:46:48.62,51:39:12.65,161.702601,51.653513,10:46:48.62,51:39:12.65,161.702601,51.653513,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,6.300000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
7,SDSS J132815.49+361715.9,202.064565,36.287773,1.088,17.0,4.14,0.49,SDSS J132815.49+361715.9,-99,1.088,-99,spec,-99,13:28:15.50,36:17:15.98,202.064565,36.287773,13:28:15.50,36:17:15.98,202.064565,36.287773,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,4.900000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
8,SDSS J133254.51+371735.5,203.227157,37.293221,2.460,19.3,1.52,0.18,SDSS J133254.51+371735.5,-99,2.460,-99,spec,-99,13:32:54.52,37:17:35.60,203.227157,37.293221,13:32:54.52,37:17:35.60,203.227157,37.293221,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,1.800000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
9,SDSS J133840.66+315936.4,204.669443,31.993451,2.944,19.6,2.33,

In [6]:
# Adding Gou+2020 here to the list of Charisi and Graham targets
# I'll be adding some comments in the notes flag about which 7 objects Guo+ honed in on

for index, row in graham2015.iterrows():
    graham2015.at[index, 'Paper(s)'] += ' ; Guo+2020'
    graham2015.at[index, 'BibCode(s)'] += ' ; 2020MNRAS.492.2910G' 
    graham2015.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stz3566'
    
#graham2015

charisi_per = ["UM 269","SDSS J005158.83-002054.1","SDSS J005453.30-003258.3","SDSS J024442.77-004223.2","2QZ J095344.7+010354","SDSS J133254.51+371735.5","SDSS J133840.66+315936.4","SDSS J134553.57+334336.0","SDSS J134556.16+343224.5","SDSS J141004.41+334945.5","SDSS J141244.09+421257.6","SDSS J142339.44+471240.8","SDSS J150900.70+175114.3","SDSS J151646.10+221724.7","SDSS J152739.97+413234.6","SDSS J153051.79+503440.1","SDSS J153251.06+335852.2","PDS 898","SDSS J160454.57+315733.5","SDSS J170942.58+342316.2","SDSS J171122.67+342658.9","SDSS J171617.49+341553.3","SDSS J171909.93+344001.3","SDSS J212939.60+004845.5","SDSS J214036.77+005210.1","SDSS J214225.29+001643.2","SDSS J214357.98+003349.6","PKS 2203-215","SDSS J231733.66+001128.3","SDSS J232135.73+173916.5","2QZ J235800.2-281429","SDSS J235928.99+170426.9","SDSS J235958.72+003345.3"]
for index, row in charisi2016.iterrows():
    if row['Name1'] in charisi_per:
        #print('True') # All info gets added properly.
        charisi2016.at[index, 'Paper(s)'] += ' ; Guo+2020'
        charisi2016.at[index, 'BibCode(s)'] += ' ; 2020MNRAS.492.2910G' 
        charisi2016.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stz3566'

charisi2016
# NOTE!!!! THERE ARE 6 TOO MANY OBJECTS INCLUDED HERE. WE WILL NEED TO FIGURE OUT WHICH OBJECTS HAD FEWER THAN
# 5 BANDS OF ARCHIVAL PHOTOMETRY!!!!!


,Name,RA,Dec,z,R-mag,Separation,Angular,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,UM269,10.832244,0.854291,0.308,17.8,NaN,NaN,UM269,-99,0.308,-99,spec,-99,0:43:19.74,0:51:15.45,10.832244,0.854291,0:43:19.74,0:51:15.45,10.832244,0.854291,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,NaN,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
1,SDSS J005158.83-002054.1,12.995131,-0.348382,1.047,19.3,2.89,0.35,SDSS J005158.83-002054.1,-99,1.047,-99,spec,-99,0:51:58.83,-0:20:54.18,12.995131,-0.348382,0:51:58.83,-0:20:54.18,12.995131,-0.348382,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,3.500000e-07,-99,Optical Periodicity,-99,Charisi+2016 ; Guo+2020,2016MNRAS.463.2145C ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stw1838 ; https:...,
2,SDSS J005453.30-003258.3,13.722104,-0.549532,0.961,19.8,1.55,0.19,SDSS J005453.30-003258.3,-99,0.961,-99,spec,-99,0:54:53.30,-0:32:58.32,13.722104,-0.549532,0:54:53.30,-0:32:58.32,13.722104,-0.549532,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,1.900000e-07,-99,Optical Periodicity,-99,Charisi+2016 ; Guo+2020,2016MNRAS.463.2145C ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stw1838 ; https:...,
3,SDSS J023050.06+005843.1,37.708607,0.978662,1.447,17.9,6.65,0.77,SDSS J023050.06+005843.1,-99,1.447,-99,spec,-99,2:30:50.07,0:58:43.18,37.708607,0.978662,2:30:50.07,0:58:43.18,37.708607,0.978662,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,7.700000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
4,SDSS J024442.77-004223.2,41.178244,-0.706457,0.628,19.7,1.77,0.25,SDSS J024442.77-004223.2,-99,0.628,-99,spec,-99,2:44:42.78,-0:42:23.24,41.178244,-0.706457,2:44:42.78,-0:42:23.24,41.178244,-0.706457,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,2.500000e-07,-99,Optical Periodicity,-99,Charisi+2016 ; Guo+2020,2016MNRAS.463.2145C ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stw1838 ; https:...,
5,2QZ J095344.7+010354,148.436691,1.065195,0.994,19.3,1.70,0.22,2QZ J095344.7+010354,-99,0.994,-99,spec,-99,9:53:44.81,1:03:54.70,148.436691,1.065195,9:53:44.81,1:03:54.70,148.436691,1.065195,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,2.200000e-07,-99,Optical Periodicity,-99,Charisi+2016 ; Guo+2020,2016MNRAS.463.2145C ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stw1838 ; https:...,
6,SDSS J104648.62+513912.6,161.702601,51.653513,0.587,18.7,4.30,0.63,SDSS J104648.62+513912.6,-99,0.587,-99,spec,-99,10:46:48.62,51:39:12.65,161.702601,51.653513,10:46:48.62,51:39:12.65,161.702601,51.653513,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,6.300000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
7,SDSS J132815.49+361715.9,202.064565,36.287773,1.088,17.0,4.14,0.49,SDSS J132815.49+361715.9,-99,1.088,-99,spec,-99,13:28:15.50,36:17:15.98,202.064565,36.287773,13:28:15.50,36:17:15.98,202.064565,36.287773,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,4.900000e-07,-99,Optical Periodicity,-99,Charisi+2016,2016MNRAS.463.2145C,https://doi.org/10.1093/mnras/stw1838,
8,SDSS J133254.51+371735.5,203.227157,37.293221,2.460,19.3,1.52,0.18,SDSS J133254.51+371735.5,-99,2.460,-99,spec,-99,13:32:54.52,37:17:35.60,203.227157,37.293221,13:32:54.52,37:17:35.60,203.227157,37.293221,J2000,-99,-99,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-1

In [7]:
print(charisi2016['Dec1_deg'].dtype)

float64


In [8]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but for now we're going to run with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2




In [9]:
tunique, tmatches, idx1, idx2 = match_tables_fib(graham2015,charisi2016,5)

# There are no matches within 5'' between the Graham+ and Charisi+ tables

the_whills = pd.concat([graham2015,charisi2016])
the_whills.reset_index(drop=True, inplace=True)

the_whills.drop(labels=['RA','Dec','R-mag','Separation','Angular','index'], axis=1, inplace=True)


In [10]:
the_whills

,Name,z,Vmed,r,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes,Coordinate_waveband,Coordinate_Source
0,UM 211,1.998,17.38,0.000,UM 211,-99,1.998,-99,spec,-99,0:12:10.9,-1:22:7.6,3.045417,-1.368778,0:12:10.9,-1:22:7.6,3.045417,-1.368778,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
1,UM 234,0.729,17.82,0.011,UM 234,-99,0.729,-99,spec,-99,0:23:3.2,+1:15:33.9,5.763333,1.259417,0:23:3.2,+1:15:33.9,5.763333,1.259417,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
2,SDSS J014350.13+141453.0,1.438,17.68,0.008,SDSS J014350.13+141453.0,-99,1.438,-99,spec,-99,1:43:50.0,+14:14:54.9,25.958333,14.248583,1:43:50.0,+14:14:54.9,25.958333,14.248583,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
3,PKS 0157+011,1.170,18.04,0.000,PKS 0157+011,-99,1.170,-99,spec,-99,2:0:3.9,+1:25:12.6,30.016250,1.420167,2:0:3.9,+1:25:12.6,30.016250,1.420167,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
4,RX J024252.3-232633,0.680,19.01,0.000,RX J024252.3-232633,-99,0.680,-99,spec,-99,2:42:51.9,-23:26:34.0,40.716250,-23.442778,2:42:51.9,-23:26:34.0,40.716250,-23.442778,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
5,US 3204,0.954,18.09,0.008,US 3204,-99,0.954,-99,spec,-99,2:49:28.9,+1:9:25.0,42.370417,1.156944,2:49:28.9,+1:9:25.0,42.370417,1.156944,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
6,CT 638,0.265,16.68,0.000,CT 638,-99,0.265,-99,spec,-99,3:18:6.5,-34:26:37.4,49.527083,-34.443722,3:18:6.5,-34:26:37.4,49.527083,-34.443722,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
7,RXS J04117+1324,0.277,16.20,0.006,RXS J04117+1324,-99,0.277,-99,spec,-99,4:11:46.9,+13:24:16.5,62.945417,13.404583,4:11:46.9,+13:24:16.5,62.945417,13.404583,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https://doi.org/10.1093/mnras/stv1726 ; https:...,,NaN,NaN
8,HS 0423+0658,0.170,15.73,0.000,HS 0423+0658,-99,0.170,-99,spec,-99,4:26:30.2,+7:5:30.3,66.625833,7.091750,4:26:30.2,+7:5:30.3,66.625833,7.091750,J2000,Optical,CRTS,Binary AGN Candidate / Binary SMBH Candidate,-100,-100,-100,-100,-100,-100,-9.900000e+01,-99,Optical Periodicity,-99,Graham+2015 ; Guo+2020,2015MNRAS.453.1562G ; 2020MNRAS.492.2910G,https:/

In [11]:
the_whills.to_csv('Periodicity_catalogs_DR1.csv', sep=',', index=False)
